In [1]:
import nidaqmx as ni
import numpy as np

In [2]:
def is_device_available(device_name):
    avail_device_names = ni.system.System().devices.device_names
    return device_name in set(dev.lower() for dev in avail_device_names)

def is_counter_available(device_handle, counter_name):
    avail_counter_names = [
        ctr.split('/')[-1] for ctr in device_handle.co_physical_chans.channel_names
    ]
    return counter_name in avail_counter_names

def is_ai_term_available(device_handle, term_name):
    avail_ai_term_names = [
        term.rsplit('/', 1)[-1].lower() for term in device_handle.ai_physical_chans.channel_names
    ]
    return term_name in avail_ai_term_names

def is_ao_term_available(device_handle, term_name):
    avail_ao_term_names = [
        term.rsplit('/', 1)[-1].lower() for term in device_handle.ao_physical_chans.channel_names
    ]
    return term_name in avail_ao_term_names

def is_pfi_term_available(device_handle, term_name):
    avail_pfi_term_names = [
        term.rsplit('/', 1)[-1].lower() for term in device_handle.terminals if 'PFI' in term
    ]
    return term_name in avail_pfi_term_names


In [ ]:
class ni_tasks_manager:

    tasks_dict = dict()
    
    def create_task(task_name):
        TD = ni_tasks_manager.tasks_dict
        if task_name in TD.keys():
            raise Exception(f'{task_name} has already benn created!')
        
        try:
            task = ni.Task(task_name)
        except:
            raise Exception(f'Fail to create task named {task_name}.')
        TD[task_name] = task        
        return task

    def remove_task(name=None, task=None):
        TD = ni_tasks_manager.tasks_dict
        if task:
            if not task in TD.values():
                raise Exception('Fail to find {task} from our record!')
            for k, t in TD.items():
                if id(task) == id(t):
                    name = k
                    break
        if name:
            try:
                task = TD.pop(name)
                try:
                    if not task.is_task_done():
                        task.stop()
                    task.close()
                    del task
                except ni.DaqError:
                    print('Fail to delete task {name}')
            except:
                raise Exception('Fail to remove task {name}')
        else:
            print('Specify which task to be removed.')

    def is_name_exist(task_name):
        return task_name in ni_tasks_manager.tasks_name_list.keys()
    
    def is_task_exist(task):
        return id(task) in [id(t) for t in ni_tasks_manager.values()]


In [2]:
from hardware.nidaq import sample_clock, analog_output_constant, analog_output_sweeper

In [3]:
device_name = 'dev2'
counter_name = 'ctr0'
ao_channels = ['ao0', 'ao1']

voltage_range = [
    [0., 10.],     # ao0
    [0., 10.],     # ao1
]

period = 0.01   # seconds
sample_rate = 1/period
DutyCycle = 0.9
frame_size = 10

clk = sample_clock(device_name, counter_name)
clk.prepare_task()



In [4]:
clk.samps_per_chan = 11

In [5]:
ao_sweep = analog_output_sweeper(
    device_name, ao_channels, voltage_range,
    clk, use_falling=True
)
ao_sweep.prepare_task()

In [8]:
clk.samps_per_chan = None
clk.update_task()

In [9]:
#ao_sweep.task.timing.samp_timing_type #= ni.constants.SampleTimingType.ON_DEMAND
clk.task.timing.samp_quant_samp_mode

<AcquisitionType.CONTINUOUS: 10123>

In [11]:
ao_sweep.start()
clk.start()


In [8]:
clk.task.is_task_done()

True

In [11]:
clk.clear_task()
